# Un exemple un peu plus conséquent

Voici un exemple inspiré de l'exercice 37 du livre de Walnut. Ici on construit un DFAO qui reconnaît le mot
$$
\mathbf{x} = y \mathcal{T}(y) \mathcal{T}^2(y) \cdots
$$
où $\mathcal{T}$ est le morphisme de Thue-Morse $a \mapsto ab, b \mapsto ba$ et $y = bab$.

## 1. Outillage avec regexp et prédicats

On commence tranquillement avec une expression régulière pour reconnaître les puissances de 2 (non exprimable dans l'arithmétique de Presburger mais pas de souci dans Büchi en base 2).

In [1]:
reg pow2 msd_2 "0*10*":


computed ~:2 states - 12ms



In [2]:
%showme pow2

On construit des paires d'entiers bien choisies pour identifier les positions clés de $\mathbf{x}$.

In [3]:
def pairSL "$pow2(y) & x=3*(y-1)":

computed ~:1 states - 2ms
computed ~:2 states - 2ms
x=(3*(y-1)):6 states - 1ms
 (pow2(y))&x=(3*(y-1))):5 states - 1ms
Total computation time: 40ms.



In [4]:
%showme pairSL

## 2. Où on a besoin d'automates en plus

Pour cette construction, on a besoin de DFAO spécifique pour Thue-Morse, des versions horlogées avec état initial variable. Pas de souci, on peut générer son propre DFAO en Walnut... mais il faut aller poser le fichier dans `Word Automata Library/`. Ce serait tellement mieux d'avoir une commande Walnut pour ça !

In [8]:
%%file ../Word Automata Library/Tclocka.txt
msd_2 msd_2
0 0
0 0 -> 0
1 0 -> 0
0 1 -> 0
1 1 -> 1
1 1
0 0 -> 1
1 0 -> 1
0 1 -> 1
1 1 -> 0

Created file '/home/jovyan/Word Automata Library/Tclocka.txt'.


In [9]:
%%file ../Word Automata Library/Tclockb.txt
msd_2 msd_2
0 1
0 0 -> 0
1 0 -> 0
0 1 -> 0
1 1 -> 1
1 0
0 0 -> 1
1 0 -> 1
0 1 -> 1
1 1 -> 0

Created file '/home/jovyan/Word Automata Library/Tclockb.txt'.


## 3. Construction du DFAO

Pour construire un DFAO sur un alphabet à $k$ lettres, il faut construire les prédicats pour reconnaître les $k-1$ lettres $1$ à $k-1$. Ici l'alphabet possède deux lettres, c'est donc un unique prédicat qui est défini :

In [10]:
def babTb "Es,l,y $pairSL(s,l) & y<l & ((x=s+y & Tclockb[y][l-1] = @1) | (x=s+l+y & Tclocka[y][l-1] = @1) | (x=s+2*l+y & Tclockb[y][l-1] = @1))":


y<l:2 states - 1ms
 (pairSL(s,l))&y<l):5 states - 0ms
  x=(s+y):2 states - 1ms
   Tclockb[y][(l-1)]=@1:5 states - 1ms
    (x=(s+y)&Tclockb[y][(l-1)]=@1):10 states - 2ms
     x=((s+l)+y):3 states - 2ms
      Tclocka[y][(l-1)]=@1:5 states - 1ms
       (x=((s+l)+y)&Tclocka[y][(l-1)]=@1):12 states - 1ms
        ((x=(s+y)&Tclockb[y][(l-1)]=@1)|(x=((s+l)+y)&Tclocka[y][(l-1)]=@1)):22 states - 6ms
         x=((s+(2*l))+y):4 states - 1ms
          Tclockb[y][(l-1)]=@1:5 states - 1ms
           (x=((s+(2*l))+y)&Tclockb[y][(l-1)]=@1):14 states - 2ms
            (((x=(s+y)&Tclockb[y][(l-1)]=@1)|(x=((s+l)+y)&Tclocka[y][(l-1)]=@1))|(x=((s+(2*l))+y)&Tclockb[y][(l-1)]=@1)):43 states - 6ms
             ((pairSL(s,l))&y<l)&(((x=(s+y)&Tclockb[y][(l-1)]=@1)|(x=((s+l)+y)&Tclocka[y][(l-1)]=@1))|(x=((s+(2*l))+y)&Tclockb[y][(l-1)]=@1))):14 states - 2ms
              (E s , l , y ((pairSL(s,l))&y<l)&(((x=(s+y)&Tclockb[y][(l-1)]=@1)|(x=((s+l)+y)&Tclocka[y][(l-1)]=@1))|(x=((s+(2*l))+y)&Tclockb[y][(l-1)]=@1)))):1

In [11]:
%showme babTb

Enfin, on combine les prédicats pour former le DFAO.

In [12]:
combine babT babTb:

In [13]:
%showme babT